# CSV-link-processing

## Introduzione

L'obiettivo della repository è quello di filtrare i link pdf trovati dal crawler per identificare i bilanci di sostenibilità relativi all'anno 2018 pubblicati dalle aziende.

## Processo

Ogni riga del file csv presenta un sito web e una lista che contiene dizionari con le informazioni relative ad ogni link.

Si controlla l'idoneità di ogni link per decidere se scartarlo o meno.

Nel processo si tiene traccia di :

1. totale dei siti web
2. totale dei siti web che hanno pubblicato
3. percentuale di questi siti sul totale
4. totale file pdf
5. numero di file pdf utili
6. percentuale dei file utili sul totale
7. profondità media alla quale si trovano questi file
8. numero di aziende che pubblicano il bilancio in homepage 

Di seguito il programma principale, che utilizza le funzioni __evaluate__ e __get_depth__, descritte sotto

In [4]:
import csv, re

def clean_pdf_list(ll):
    #keep only the link and the source (needed to get depth later)
    ret = []
    for l in ll:
        k = {}
        k['pdfUrl'] = l[0]['pdfUrl']
        k['sourcePageUrl'] = l[0]['sourcePageUrl']
        ret.append(k)

    # remove #page= and GET arguments
    for r in ret:
        if "#page=" in r['pdfUrl']:
            s = r['pdfUrl']
            r['pdfUrl'] = s[:s.find("#page=")]
            s = r['pdfUrl']
            i = s.find("?") 
            j = s.find(".pdf")
            if i > 0 and j < i:
                r['pdfUrl'] = s[:s.find(".pdf")+4]
    
    #remove duplicates
    s = set()
    for d in ret:
        s.add(d['pdfUrl'])
    no_dup = []
    for r in ret:
        if r['pdfUrl'] in s:
            no_dup.append(r)
            s.remove(r['pdfUrl'])

    return no_dup

def get_stats(s):
    return ("Total websites,%d\nWebsites who \
published,%d\nPublished percentage,%f\nTotal \
links,%d\nProbable sustainability pdfs links,%d\n\
Useful pdfs percentage,%f\nAverage depth,%f\n\
Pdfs in homepage,%d" %(s[0], s[1], (s[1]/s[0])*100, s[2], s[3], (s[3]/s[2])*100, s[4], s[5]))


csv_file = "small.csv"
ret = {}
csv.field_size_limit(100000000)
with open(csv_file) as csv_file:    
    csv_reader = csv.reader(csv_file, delimiter=',')
    total = useful = depth_sum = homelinks = 0
    counter = -1
    pdf_dump = []
    for row in csv_reader:
        counter += 1            
        if counter == 0 : continue
        pdf_dump = eval(row[1])
        total += len(pdf_dump)
        pdf_dump = [k for k in [(l,evaluate(l)) for l in pdf_dump] if k[1][0]]
        pdf_dump = clean_pdf_list(pdf_dump)
        for l in pdf_dump:
            d = get_depth(l)
            if d == 1:
                homelinks += 1
            depth_sum += d

        useful += len(pdf_dump)
        if pdf_dump:
            ret[row[0]] = [l['pdfUrl'] for l in pdf_dump] 

stats = (counter, len(ret), total, useful, depth_sum/useful, homelinks)

print("Useful links: \n%s\n" %ret)
print("Stats: \n%s" %get_stats(stats).replace(",", ": "))

Useful links: 
{'www.iav.com': ['https://www.iav.com/app/uploads/2019/02/IAV_Sustainability_Report_2018.pdf']}

Stats: 
Total websites: 7
Websites who published: 1
Published percentage: 14.285714
Total links: 62
Probable sustainability pdfs links: 1
Useful pdfs percentage: 1.612903
Average depth: 3.000000
Pdfs in homepage: 0


## Get depth

Funzione che resituisce la profondità alla quale si trova l'anchor del file pdf.

Scarta eventuali livelli inutili, rappresentati ad esempio dalla lingua della pagina, un _/home_ , _/homepage_ , etc..

Il _set di livelli_ da non contare è molto più grande, per semplicità è ridotto nell'esempio seguente.

In [2]:
NOT_COUNT_SET = set(["it", "en", "de", "home", "homepage"])

def get_depth(l):
    s = l["sourcePageUrl"]
    #"remove https://"
    ss = s[s.find("://")+3:]
    #remove last / if present
    if ss[-1] == "/":
        ss = ss[:-1]
    # count levels
    d = ss.count("/") +1
    k = d
    # remove unnecessary levels (eg: /home /en /it)
    for i in range(d):
        if ss[:ss.find("/")] in NOT_COUNT_SET:
            k -= 1
        ss = ss[ss.find("/")+1:]
    return k

## Evaluate

Funzione che si occupa di determinare se un file è ideoneo oppure no.

In [3]:
EVALUATION_THRESHOLD = 40

def evaluate(link):
    score = 0
    filename = link['pdfUrl'].split("/")[-1].casefold()

    url = link['pdfUrl'][link['pdfUrl'].find("://")+3:]
    url = url[url.find("/")+1:-len(filename)]
    anchor = link['anchor'].casefold()

    if "sostenibilit" in filename or "sostenibilit" in anchor or \
       "sustainability" in filename or "sustainability" in anchor: 
        score += 40

    if "ambient" in filename or "ambient" in anchor or \
       "environment" in filename or "environment" in anchor:
        score += 20
    
    if "bilancio" in filename or "bilancio" in anchor or \
       "balance" in filename or "balance" in anchor:
        score += 20

    if "rapporto" in filename or "rapporto" in anchor or \
       "report" in filename or "report" in anchor:
        score += 20

    if "sostenibilit" in url or "sustainability" in url: 
        score += 10

    if "ambient" in url or "environment" in url: 
        score += 10

    year = int("2018" in filename) + int("2018" in url) + int("2018" in anchor)

    if re.match(r'(.*)20[0-2]([0-7]|[9])(.*)', filename) and not re.match(r'(.*)18(.*)', filename):
        score = 0
        
    return score >= EVALUATION_THRESHOLD and year > 0, score, year

### Score

Osservando i blocchi _if_ della funzione si può notare che:
1. la presenza di __sostenibilità__ nel filename oppure nell'anchor incrementa lo score di molto, rendendola di fatto condizione sufficiente, se presente anche un'occorrenza di __2018__ , per la valutazione positiva di un pdf
2. le parole __ambiente__ , __bilancio__ e __rapporto__ hanno meno importanza di __sostenibilità__ , aumentando lo score della metà
3. la presenza di _sostenibilità_ o _ambiente_ nell'url contribuisce in maniera marginale rispetto al resto

### Anno

Dopo aver valutato lo score di un link, si passa alla ricerca di occorrenze di __2018__.
Si conta quante volte compare nel filename, anchor e url.

Infine si passa alla ricerca di file non idonei per quanto riguarda l'anno del bilancio di sostenibilità.

Vi sono infatti casi in cui compaiono uno o più __2018__, ma il file non riguarda quell'anno, è il caso ad esempio di siti gestiti con Wordpress in cui nel 2018 è stato caricato un bilancio del 2017, in quel caso ci sarà un url del tipo /2018/mese/giorno/filename.

Si vogliono però tenere i file che riguardano l'anno 2018 nonostante sia presente un 17, o un 19.
È il caso di file come _bilancio sostenibilità 17-18.pdf_.

### Valutazione

La condizione necessaria per considerare un file idoneo oppure no, tiene in considerazione lo score e le occorrenze dell'anno su cui ci si concentra:
Lo score deve essere almeno di 40, e deve essere essere presente almeno un occorrenza di 2018.

È  stato scelto uno score di 40 per includere casi in cui è presente almeno _sostenibilità_ nel filename o nell'anchor, oppure una combinazione di _bilancio_ / _rapporto_ / _ambiente_.

### Condizioni di valutazione alternative

Alcune condizioni alternative sono le seguenti:

```
1. score >= EVALUATION_THRESHOLD and year > 0
2. score >= EVALUATION_THRESHOLD and "2018" in filename
3. score >= EVALUATION_THRESHOLD and "2018" in anchor
4. score >= EVALUATION_THRESHOLD and ("2018" in filename or "2018" in anchor)
5. score >= EVALUATION_THRESHOLD
```

Si può notare sempre lo stesso andamento per quanto riguarda le differenze tra le condizioni, mentre la threshold alza o abbassa l'andamento complessivo.

## Test valutazioni

A seguire i risultati della funzione con due valori di threshold, 40 e 60.
L'ultima condizione non tiene conto che il pdf sia del 2018, si può notare che impostare qualsiasi tipo di restrizione sull'anno riduce di molto il numero di bilanci trovati.

### Numero di siti web che hanno pubblicato il bilancio
La stima iniziale di siti che hanno pubblicato, ovvero circa 4000, fatta prima di perferzionare la funzione __evaluation__ era evidentemente errata, infatti le parole chiave erano ricercate nell'url completo, compreso di dominio, senza rimuovere i duplicati e senza dare più importanza alla keyword _sostenibilità_.
Il numero di riscontri ora è chiaramente minore, ma si apprezza una pertinenza con il tema della sostenibilità molto più elevata.
<img src="graphs/0.png" style="width: 400px;">

### Percentuale nel totale
<img src="graphs/1.png" style="width: 400px;">

### Numero di pdf utili
Si nota che il numero di pdf è sempre maggiore del numero di siti web che pubblicano, questo perchè circa il __18%__ dei siti pubblica un numero che varia da 2 a 4 di pdf che sono ritenuti utili.
Può succedere perchè in alcuni casi sono traduzioni dello stesso file, o in alcuni casi un summary, accompagnato dal file completo.
Ritengo che si possa accettare uno scenario del genere per il momento, per poi gestirlo nella successiva parte di analisi semantica del testo, andando ad "unire" le entità trovate dai vari file.
<img src="graphs/2.png" style="width: 400px;">

### Percentuale di file pdf utili
<img src="graphs/3.png" style="width: 400px;">

### Profondità media dei file
<img src="graphs/4.png" style="width: 400px;">

### Link presenti in homepage
<img src="graphs/5.png" style="width: 400px;">